迭代过程中，significance的变化

$|\frac {t(n,i)-t(n-1)}{t(n-1)}|$ 在单机情况下, 退化为$|\frac {t(n)-t(n-1)}{t(n-1)}|$

$f(n,i)-f(n-1,i)-...-f(n-k,i),f(n,i)=|\frac {t(n,i)-t(n-1)}{t(n-1)}|,k表示要向前考虑的迭代轮数，是一个超参数$退化为$f(n)-f(n-1)-...-f(n-k),f(n)=|\frac {t(n)-t(n-1)}{t(n-1)}|,k表示要向前考虑的迭代轮数，是一个超参数$


训练过程是用CNN训练FEMNIST(新数据集)

In [ ]:
import torch
import torchvision
from torch import nn
import random
import numpy as np
from d2l import torch as d2l


# 通用函数

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)


In [ ]:
def func(net_1,net_2):
    """
        输入更新前后的网络f(n) f(n-1)
        对各个参数应用 (w(n)-w(n-1))/w(n-1), 最后加权平均
    """
    net_1.parameters()
    net_2.parameters()

    return 

# CNN

In [ ]:
class FedAvgCNN(nn.Module):
    def __init__(self,dropout=0.5) -> None:
        super().__init__()
        self.conv2d_1 = nn.Conv2d(1,32,(5,5))
        self.max_pool = nn.MaxPool2d((2,2))
        self.conv2d_2 = nn.Conv2d(32,64,(5,5))
        self.flat = nn.Flatten()
        self.fc = nn.Linear(1024,512)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        self.linear = nn.Linear(512,10)

    def forward(self,x):
        x = self.conv2d_1(x) # 32,24,24
        x = self.relu(x) # 32,24,24
        x = self.max_pool(x) # 32,12,12

        x = self.conv2d_2(x) # 32,12,12
        x = self.relu(x) # 32,12,12
        x = self.max_pool(x) # 32,6,6

        x = self.flat(x) # 1152
        x = self.fc(x) # 512
        x = self.relu(x) # 512
        x = self.dropout(x) # 512

        x = self.softmax(x) # 512

        x = self.linear(x) # 10

        return x

    def init_params(self, seed):
        set_seed(seed)
        for layer in self.children():
            if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
                # 参数初始化方法一般与激活函数有关
                # Relu-kaming
                # sigmoid-xavier
                nn.init.kaiming_normal_(layer.weight.data)
                nn.init.zeros_(layer.bias.data)

# 加载数据

In [ ]:
def load_data():
    pass

# 训练

## 常量

In [ ]:
epoch = 50
lr = 1e-3
momentum = 0.9

In [ ]:
device = d2l.try_gpu()
device

In [ ]:
cnn = FedAvgCNN()
cnn = cnn.to(device)

In [ ]:
loss_func = torch.nn.CrossEntropyLoss()
optim = torch.optim.SGD(cnn.parameters(),lr=lr,momentum=momentum)

In [ ]:
train_dataloader,test_dataloader = load_data()

In [ ]:
animator = d2l.Animator('iter',['train_loss','train_acc','test_acc'])

for e in range(epoch):
    cnn.train()

    inner_accum = d2l.Accumulator(3)
    for x,y in train_dataloader:
        x = x.to(device)
        y = y.to(device)
        y_hat = cnn(y)
        loss = loss_func(y_hat,y)
        optim.zero_grad()
        loss.backward()
        optim.step()

        train_acc = d2l.accuracy(y_hat,y)
        n = len(x)
        inner_accum.add(loss*n,train_acc*n,n)

    cnn.eval()
    test_acc = d2l.evaluate_accuracy_gpu(cnn,test_dataloader,device)
    animator.add(inner_accum[0]/inner_accum[-1],inner_accum[1]/inner_accum[-1],test_acc)

